# Attention
注意力机制（Attention）

## 注意力机制的动机
传统的序列模型（如RNN和LSTM）在处理长序列时，容易出现以下问题：
- **信息瓶颈**：所有输入信息被压缩成一个固定长度的隐状态向量（即上下文向量），难以捕获序列中远距离的信息。
- **长距离依赖问题**：模型对输入序列较远部分的信息捕获能力减弱。

注意力机制通过允许模型对输入序列中的每个位置进行加权关注，解决了这些问题。它使得模型可以动态选择关注哪些输入特征，从而提高性能。


## 注意力机制的基本原理
注意力机制的核心思想是：根据当前任务的需求，计算输入中每个元素的重要性（权重），并将其加权求和用于生成输出。

假设有以下输入：
- **Query** (`q`)：当前任务的查询向量，用来决定需要关注什么。
- **Key** (`k`)：输入序列中的特征，用来计算与查询的相关性。
- **Value** (`v`)：输入序列中的信息，表示实际的内容。

### 计算过程：
1. **相关性评分**：
- 计算 `q` 和 `k` 的相似度（相关性），通常使用点积或其他相似性度量函数：
$
\text{score}(q, k_i) = q \cdot k_i
$

2. **权重归一化**：
- 对所有的 `score(q, k)` 进行归一化，常用 **Softmax** 函数，将其转换为概率分布：
$
\alpha_i = \frac{\exp(\text{score}(q, k_i))}{\sum_{j} \exp(\text{score}(q, k_j))}
$

其中，\(\alpha_i\) 是输入 \(i\) 对于查询 \(q\) 的注意力权重。

3. **加权求和**：
- 使用注意力权重对 `v` 进行加权求和，得到输出向量：
$
\text{Attention}(q, K, V) = \sum_{i} \alpha_i v_i
$

最终，注意力机制的输出是对输入序列的一个加权表示，其中权重反映了输入对当前任务的相关性。



In [2]:
import torch
import torch.nn as nn

class GeneralAttention(nn.Module):
    def __init__(self, d_model, d_k=None, d_v=None):
        """
        初始化注意力模块。
        
        Args:
            d_model: 输入的维度（嵌入维度）。
            d_k: 键和查询的维度，如果为None，则默认与d_model相同。
            d_v: 值的维度，如果为None，则默认与d_model相同。
        """
        super().__init__()
        self.d_model = d_model
        self.d_k = d_k if d_k is not None else d_model
        self.d_v = d_v if d_v is not None else d_model
        
        # 初始化线性变换矩阵
        self.W_q = nn.Linear(d_model, self.d_k)
        self.W_k = nn.Linear(d_model, self.d_k)
        self.W_v = nn.Linear(d_model, self.d_v)
        
    def forward(self, query, key_value):
        """
        前向传播函数。
        
        Args:
            query: (batch_size, seq_len_q, d_model) - 查询序列。
            key_value: (batch_size, seq_len_kv, d_model) - 键值对序列。
            
        Returns:
            output: (batch_size, seq_len_q, d_v) - 注意力输出。
        """
        # 计算查询、键和值
        Q = self.W_q(query)         # (batch_size, seq_len_q, d_k)
        K = self.W_k(key_value)     # (batch_size, seq_len_kv, d_k)
        V = self.W_v(key_value)     # (batch_size, seq_len_kv, d_v)
        
        # 计算注意力分数
        scores = torch.bmm(Q, K.transpose(-2, -1))  # 点积：(batch_size, seq_len_q, seq_len_kv)
        scores = scores / (self.d_k ** 0.5)         # 缩放
        
        # 应用Softmax得到注意力权重
        weights = torch.softmax(scores, dim=-1)     # (batch_size, seq_len_q, seq_len_kv)
        
        # 加权求和得到输出
        output = torch.bmm(weights, V)              # (batch_size, seq_len_q, d_v)
        
        return output
    

# 初始化模型参数
d_model = 512        # 输入维度
d_k = 64             # 键和查询的维度
d_v = 64             # 值的维度

attention = GeneralAttention(d_model=d_model, d_k=d_k, d_v=d_v)

# 创建输入张量（示例）
batch_size = 32
seq_len_q = 10        # 查询序列长度
seq_len_kv = 20       # 键值对序列长度

query = torch.randn(batch_size, seq_len_q, d_model)
key_value = torch.randn(batch_size, seq_len_kv, d_model)

# 前向传播
output = attention(query, key_value)  # 输出形状：(32, 10, 64)

print(output.shape)  # torch.Size([32, 10, 64])

torch.Size([32, 10, 64])


## 自注意力（Self-Attention）
- 自注意力是一种特殊形式的注意力机制，其中 Query、Key 和 Value 全部来自同一个序列。
- 每个位置的向量可以关注序列中的其他位置，从而捕获全局依赖关系。
- 自注意力是 Transformer 模型的核心。

In [ ]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_model, d_k, d_v):
        super(SelfAttention, self).__init__()
        self.d_model = d_model  # 模型维度
        self.d_k = d_k          # 键的维度
        self.d_v = d_v          # 值的维度
        
        # 初始化查询、键、值的线性变换层
        self.W_q = nn.Linear(d_model, d_k)
        self.W_k = nn.Linear(d_model, d_k)
        self.W_v = nn.Linear(d_model, d_v)
        
    def forward(self, x):
        """
        前向传播函数，计算自注意力输出。
        
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, d_model)
            
        Returns:
            输出张量，形状为 (batch_size, seq_len, d_v)
        """
        # 计算查询、键、值向量
        Q = self.W_q(x)  # (batch_size, seq_len, d_k)
        K = self.W_k(x)  # (batch_size, seq_len, d_k)
        V = self.W_v(x)  # (batch_size, seq_len, d_v)
        
        # 计算注意力分数矩阵
        attention_scores = torch.bmm(Q, K.transpose(1,2))  # 点积，形状为 (batch_size, seq_len, seq_len)
        attention_scores = attention_scores / (self.d_k ** 0.5)  # 缩放
        
        # 应用Softmax函数得到注意力权重
        attention_weights = torch.softmax(attention_scores, dim=2)  # (batch_size, seq_len, seq_len)
        
        # 计算加权求和的值向量
        output = torch.bmm(attention_weights, V)  # (batch_size, seq_len, d_v)
        
        return output

# 示例使用
if __name__ == "__main__":
    batch_size = 2
    seq_len = 3
    d_model = 4
    d_k = 5
    d_v = 6
    
    x = torch.randn(batch_size, seq_len, d_model)
    attention = SelfAttention(d_model, d_k, d_v)
    
    output = attention(x)
    print("输出形状:", output.shape)  # 应该是 (2, 3, 6)

## 多头注意力机制（Multi-Head Attention）
多头注意力（Multi-Head Attention, MHA）是 Transformer 中提出的一种改进版本。它的核心思想是：通过多个注意力头（head）来捕获输入序列中不同层次的相关性。

### 计算过程：
1. 将输入的 `q`、`k`、`v` 投影到多个子空间，生成多组 `q_i`、`k_i`、`v_i`。
2. 对每组子空间执行独立的注意力计算：
$
\text{head}_i = \text{Attention}(q_i, k_i, v_i)
$
3. 将所有注意力头的输出拼接起来，并通过一个线性变换得到最终输出：
$
\text{MHA}(Q, K, V) = \text{Concat}(\text{head}_1, \dots, \text{head}_h)W_o
$

其中 \(W_o\) 是线性变换的可学习参数。

多头注意力的优点：
- 每个头可以关注输入序列中不同的部分，捕获多样性特征。
- 提高了模型的表达能力。

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, d_k, d_v, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads  # 头的数量
        self.d_model = d_model      # 模型维度
        self.d_k = d_k              # 键的维度
        self.d_v = d_v              # 值的维度
        
        # 确保d_k和d_v能被num_heads整除
        assert d_k % num_heads == 0, "d_k必须是num_heads的整数倍"
        assert d_v % num_heads == 0, "d_v必须是num_heads的整数倍"
        
        self.d_k_per_head = d_k // num_heads  # 每个头的键维度
        self.d_v_per_head = d_v // num_heads  # 每个头的值维度
        
        # 初始化每个头的查询、键、值变换矩阵
        self.W_q = nn.Linear(d_model, d_k)
        self.W_k = nn.Linear(d_model, d_k)
        self.W_v = nn.Linear(d_model, d_v)
        
        # 最终的线性层用于整合多头输出
        self.W_o = nn.Linear(d_v, d_model)

    def forward(self, x):
        """
        前向传播函数，计算多头注意力输出。
        
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, d_model)
            
        Returns:
            输出张量，形状为 (batch_size, seq_len, d_model)
        """
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        # 计算查询、键、值向量
        Q = self.W_q(x)  # (batch_size, seq_len, d_k)
        K = self.W_k(x)  # (batch_size, seq_len, d_k)
        V = self.W_v(x)  # (batch_size, seq_len, d_v)
        
        # 将查询、键、值向量分割为num_heads个头
        Q = Q.view(batch_size, seq_len, self.num_heads, self.d_k_per_head).transpose(1,2)
        K = K.view(batch_size, seq_len, self.num_heads, self.d_k_per_head).transpose(1,2)
        V = V.view(batch_size, seq_len, self.num_heads, self.d_v_per_head).transpose(1,2)
        
        # 计算每个头的注意力分数矩阵
        attention_scores = torch.matmul(Q, K.transpose(-2,-1))  # 点积，形状为 (batch_size, num_heads, seq_len, seq_len)
        attention_scores = attention_scores / (self.d_k_per_head ** 0.5)  # 缩放
        
        # 应用Softmax函数得到注意力权重
        attention_weights = torch.softmax(attention_scores, dim=-1)  # (batch_size, num_heads, seq_len, seq_len)
        
        # 计算每个头的加权求和值向量
        output = torch.matmul(attention_weights, V)  # (batch_size, num_heads, seq_len, d_v_per_head)
        
        # 将所有头的结果拼接起来
        output = output.transpose(1,2).contiguous().view(batch_size, seq_len, self.d_v)
        
        # 通过线性层整合多头输出
        output = self.W_o(output)  # (batch_size, seq_len, d_model)
        
        return output

# 示例使用
if __name__ == "__main__":
    batch_size = 2
    seq_len = 3
    d_model = 4
    d_k = 8
    d_v = 8
    num_heads = 4
    
    x = torch.randn(batch_size, seq_len, d_model)
    attention = MultiHeadAttention(d_model, d_k, d_v, num_heads)
    
    output = attention(x)
    print("输出形状:", output.shape)  # 应该是 (2, 3, 4)

输出形状: torch.Size([2, 3, 4])


## 总结

注意力机制的计算复杂度为 $O(n^2)$，其中 $n$ 为输入序列的长度，这是因为在计算注意力权重矩阵时，每个查询都要与所有键进行点积运算。尽管多头注意力会引入额外的并行计算，但总体复杂度仍保持在 $O(n^2)$。